In [2]:
import numpy as np

In [30]:
def lu_decomposition_inplace(A, dU=None):
    n = A.shape[0]
    
    if dU is None:
        dU = np.ones(n)
    
    A[0, 0] = A[0, 0] / dU[0]  # L[0,0]
    
    for j in range(1, n):
        A[0, j] = A[0, j] / A[0, 0]  # U[0,j] = A[0,j] / L[0,0]
    
    for i in range(1, n):
        A[i, 0] = A[i, 0] / dU[0]  # L[i,0] = A[i,0] / U[0,0]
    
    for p in range(1, n):
        sum_l = 0
        for k in range(p):
            sum_l += A[p, k] * A[k, p]  # L[p,k] * U[k,p]
        A[p, p] = (A[p, p] - sum_l) / dU[p]  # L[p,p]

        for j in range(p + 1, n):
            sum_u = 0
            for k in range(p):
                sum_u += A[p, k] * A[k, j]  # L[p,k] * U[k,j]
            A[p, j] = (A[p, j] - sum_u) / A[p, p]  # U[p,j]

        for i in range(p + 1, n):
            sum_l = 0
            for k in range(p):
                sum_l += A[i, k] * A[k, p]  # L[i,k] * U[k,p]
            A[i, p] = (A[i, p] - sum_l) / dU[p]  # L[i,p]
    
    return A, dU

# After the decomposition, A = LU and det(A) = det(L) * det(U) 
# Therefore det(A) = det(L) * det(U) = product of the diagonal elements of U and L
def find_determinant(A, dU):

    return np.prod([A[i][i] for i in range(A.shape[0])]) * np.prod([dU[i] for i in range(len(dU))])


# Given A = LU we have to solve the system
# Ax = b.
# Since A = LU we can solve for
# LUx = b

# Using substitution we can substitute Ux as y , therefore we will need to solve 2 triangular systems

# Ly = b
# and 
# Ux = y

# Then we modify to solve the system with the inplace solution of the LU 
def solve_system(A, dU, b):
    n = A.shape[0]
    
    # Ly = b (fofward substitution)
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i]
        for j in range(i):
            y[i] -= A[i, j] * y[j]
        y[i] /= A[i, i]
    
    # Ux = y (backward substitution)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for j in range(i+1, n):
            x[i] -= A[i, j] * x[j]
        x[i] /= dU[i]
    
    return x

# Verify the solution using the norm of the difference between the product of A and x minus b
def verify_solution(A, x, b):
    return (np.sqrt(np.sum((np.dot(A, x) - b)**2)) < 1e-9)

# Solve using numpy
def solve_system_numpy(A, b):
    return np.linalg.solve(A, b)


In [ ]:
import numpy as np

n = 3

# A = np.array([
#     [4, 2, 3],
#     [2, 7, 5.5],
#     [6, 3, 12.5]
# ])

# b = [21.6, 33.6, 51.6]

# dU = [2, 3, 4]


A = np.array([
    [2.5, 2, 2],
    [-5, -2, -3],
    [5, 6, 6.5]
])

b = [2, -6, 2]

dU = [1, 1, 1]

Ainit = A.copy()

valid = True

for k in range(1, n+1):
    Ak = A[:k, :k]
    det = np.linalg.det(Ak)
    if abs(det) < 1e-8:
        valid = False
        print("Nu se poate calcula o descompunere LU")
        break

if valid:
    ALU, newDu = lu_decomposition_inplace(A, dU)
    print("LU Decomposition of A =\n", ALU)
    
    x = solve_system(ALU, newDu, b)
    xNumPy = solve_system_numpy(Ainit, b)
    

    print("Determinant using LU decomposition:\n", find_determinant(ALU, dU))
    print("Solution to the system Ax = b\n x =", x)
    print("The solution is valid:", verify_solution(Ainit, x, b))
    print("Solution to the system Ax = b using solely numpy\n x =", xNumPy)

    


LU Decomposition of A =
 [[2.  1.  1.5]
 [1.  2.  2. ]
 [3.  0.  2. ]]
Determinant using LU decomposition:
 192.0
Solution to the system Ax = b
 x = [2.5 2.2 2.4]
The solution is valid: True
Solution to the system Ax = b using solely numpy
 x = [2.5 2.2 2.4]
